In [1]:
# | default_exp chatapp

In [2]:
# | export

from typing import Optional, Callable, List, Dict, Any
from dataclasses import dataclass, field
from fasthtml.common import *
import asyncio
import inspect

from pylogue.session import SessionManager, InMemorySessionManager, ChatSession, Message
from pylogue.service import ChatService, Responder, ErrorHandler
from pylogue.renderer import ChatRenderer
from pylogue.cards import ChatCard

## ChatApp - Full Dependency Injection Architecture

This module provides a complete chat application with:
- FastHTML + WebSocket integration
- Spinner/loading states
- Full dependency injection
- Customizable components

In [ ]:
# | export


@dataclass
class ChatAppConfig:
    """Configuration for ChatApp."""

    # App metadata
    app_title: str = "Chat Application"
    page_title: str = "Chat"

    # Initial messages
    initial_messages_factory: Optional[Callable[[], List[Message]]] = None

    # Styling
    bg_color: str = "#1a1a1a"
    header_style: str = "text-align: center; padding: 1em; color: white;"
    container_style: Optional[str] = None

    # WebSocket settings
    ws_endpoint: str = "/ws"
    chat_endpoint: str = "/chat"

    # FastHTML extensions and headers
    markdown_enabled: bool = True
    syntax_highlighting: bool = True
    highlight_langs: List[str] = field(
        default_factory=lambda: ["python", "javascript", "html", "css"]
    )

    # Spinner configuration
    spinner_css: Optional[str] = None

    def get_default_initial_messages(self) -> List[Message]:
        """Get default initial messages."""
        return [
            Message(role="User", content="Hi"),
            Message(
                role="Assistant",
                content="Hello! I'm your helpful assistant. How can I assist you today?",
            ),
        ]

    def get_spinner_style(self) -> str:
        """Get spinner CSS styles."""
        if self.spinner_css:
            return self.spinner_css

        return """
        .spinner {
            display: inline-block;
            width: 20px;
            height: 20px;
            border: 3px solid rgba(255, 255, 255, 0.3);
            border-top-color: #fff;
            border-radius: 50%;
            animation: spin 1s linear infinite;
        }
        
        @keyframes spin {
            to { transform: rotate(360deg); }
        }
        """

In [ ]:
# | export


class ChatApp:
    """Main chat application composing all components."""

    def __init__(
        self,
        session_manager: SessionManager,
        chat_service: ChatService,
        renderer: ChatRenderer,
        config: Optional[ChatAppConfig] = None,
    ):
        """
        Initialize ChatApp with dependency injection.

        Args:
            session_manager: Manages chat sessions
            chat_service: Handles message processing
            renderer: Renders UI components
            config: Application configuration
        """
        self.session_manager = session_manager
        self.chat_service = chat_service
        self.renderer = renderer
        self.config = config or ChatAppConfig()

        # Create FastHTML app
        self.app = self._create_fasthtml_app()
        self._register_routes()

    def _create_fasthtml_app(self) -> FastHTML:
        """Create and configure FastHTML application."""
        headers = []

        # Add markdown support
        if self.config.markdown_enabled:
            headers.append(MarkdownJS())

        # Add syntax highlighting
        if self.config.syntax_highlighting:
            headers.append(HighlightJS(langs=self.config.highlight_langs))

        # Add spinner styles
        headers.append(Style(self.config.get_spinner_style()))

        return FastHTML(exts="ws", hdrs=tuple(headers))

    def _get_initial_messages(self) -> List[Message]:
        """Get initial messages for new sessions."""
        if self.config.initial_messages_factory:
            return self.config.initial_messages_factory()
        return self.config.get_default_initial_messages()

    def _register_routes(self):
        """Register HTTP and WebSocket routes."""

        @self.app.route(self.config.chat_endpoint)
        def home():
            """Main chat interface."""
            initial_messages = self._get_initial_messages()

            container_style = self.config.container_style or (
                f"font-family: monospace, sans-serif; margin: 0; padding: 0; "
                f"background: {self.config.bg_color}; min-height: 100vh;"
            )

            return (
                Title(self.config.page_title),
                Div(
                    H1(self.config.app_title, style=self.config.header_style),
                    self.renderer.render_messages(initial_messages),
                    self.renderer.render_form(),
                    style=container_style,
                ),
            )

        # Register WebSocket route with message handler
        @self.app.ws(
            self.config.ws_endpoint, conn=self._on_connect, disconn=self._on_disconnect
        )
        async def ws_handler(msg: str, send, ws):
            """WebSocket message handler."""
            await self._handle_websocket_message(msg, send, ws)

    def _on_connect(self, ws, send):
        """Handle WebSocket connection."""
        session_id = str(id(ws))
        initial_messages = self._get_initial_messages()
        self.session_manager.create_session(
            session_id=session_id, initial_messages=initial_messages
        )

    def _on_disconnect(self, ws):
        """Handle WebSocket disconnection."""
        session_id = str(id(ws))
        self.session_manager.delete_session(session_id)

    async def _handle_websocket_message(self, msg: str, send, ws):
        """
        Handle incoming WebSocket message with streaming support:
        1. Add user message
        2. Add empty assistant message for streaming
        3. Stream response tokens and update message progressively
        4. Clear input
        """
        session_id = str(id(ws))
        session = self.session_manager.get_session(session_id)

        if session is None:
            # Recreate session if somehow missing
            initial_messages = self._get_initial_messages()
            session = self.session_manager.create_session(
                session_id=session_id, initial_messages=initial_messages
            )

        # Step 1: Add user message and render
        session.add_message("User", msg)
        await send(self.renderer.render_messages(session.get_messages()))

        # Step 2: Add empty assistant message for streaming
        assistant_msg = session.add_message("Assistant", "", pending=False)

        # Step 3: Stream response and update progressively
        try:
            response_chunks = []
            chunk_count = 0
            async for chunk in self.chat_service.process_message_stream(msg, session):
                response_chunks.append(chunk)
                chunk_count += 1
                # Update the assistant message with accumulated response
                full_response = "".join(response_chunks)
                session.update_message(
                    assistant_msg.id, content=full_response, pending=False
                )
                # Send updated message list to UI
                print(f"📤 Sending chunk #{chunk_count}: {repr(chunk)}")  # Debug
                await send(self.renderer.render_messages(session.get_messages()))

        except Exception as e:
            # Handle errors
            error_msg = f"Error: {str(e)}"
            session.update_message(assistant_msg.id, content=error_msg, pending=False)
            await send(self.renderer.render_messages(session.get_messages()))

        # Step 4: Clear input field
        await send(self.renderer.render_input())

    def run(
        self, host: str = "0.0.0.0", port: int = 5001, reload: bool = False, **kwargs
    ):
        """
        Run the chat application.

        Args:
            host: Host to bind to
            port: Port to bind to
            reload: Enable auto-reload on code changes
            **kwargs: Additional arguments for uvicorn.run()
        """
        import uvicorn

        print(f'Link: http://{"localhost" if host=="0.0.0.0" else host}:{port}')
        uvicorn.run(self.app, host=host, port=port, reload=reload, **kwargs)

    def get_app(self) -> FastHTML:
        """Get the underlying FastHTML app for advanced usage."""
        return self.app

## Factory Functions for Easy Setup

In [5]:
# | export


def create_default_chat_app(
    responder: Responder,
    config: Optional[ChatAppConfig] = None,
    card: Optional[ChatCard] = None,
    session_manager: Optional[SessionManager] = None,
    error_handler: Optional[ErrorHandler] = None,
    context_provider: Optional[Callable] = None,
) -> ChatApp:
    """
    Factory function to create a ChatApp with sensible defaults.

    Args:
        responder: The AI/LLM responder function
        config: Optional app configuration
        card: Optional custom ChatCard for styling
        session_manager: Optional custom session manager
        error_handler: Optional custom error handler
        context_provider: Optional context provider for responder

    Returns:
        Configured ChatApp instance
    """
    # Use defaults if not provided
    session_manager = session_manager or InMemorySessionManager()
    chat_service = ChatService(
        responder=responder,
        error_handler=error_handler,
        context_provider=context_provider,
    )
    renderer = ChatRenderer(card=card)
    config = config or ChatAppConfig()

    return ChatApp(
        session_manager=session_manager,
        chat_service=chat_service,
        renderer=renderer,
        config=config,
    )

## Example: Creating a Chat App

In [6]:
# | export


# Example responder
async def example_responder(message: str, context=None) -> str:
    """Example echo responder with latency."""
    await asyncio.sleep(0.5)  # Simulate processing time
    return f"Echo: {message}"

In [7]:
# Example 1: Minimal setup with defaults
app = create_default_chat_app(responder=example_responder)

# To run: app.run()
print("✅ Created default chat app")

✅ Created default chat app


In [8]:
# Example 2: Custom configuration
custom_config = ChatAppConfig(
    app_title="My AI Assistant",
    page_title="AI Chat",
    bg_color="#0a0a0a",
    markdown_enabled=True,
    syntax_highlighting=True,
)

# Custom card styling
custom_card = ChatCard(
    user_color="#2d4a3e",
    assistant_color="#3d2a4a",
    user_emoji="👤",
    assistant_emoji="🤖",
    font_size="1.2em",
)

app = create_default_chat_app(
    responder=example_responder, config=custom_config, card=custom_card
)

print("✅ Created custom styled chat app")

✅ Created custom styled chat app


In [9]:
# Example 3: Full DI with custom components
from pylogue.service import ContextAwareResponder

# Custom responder that uses history
context_responder = ContextAwareResponder(max_history=10)


# Custom context provider
def history_context_provider(session: ChatSession):
    """Provide last N messages as context."""
    messages = session.get_messages()
    return messages[-5:] if len(messages) > 5 else messages


# Custom session manager (could be Redis, DB, etc.)
custom_session_manager = InMemorySessionManager()

# Assemble with full DI
chat_service = ChatService(
    responder=context_responder, context_provider=history_context_provider
)

renderer = ChatRenderer(card=ChatCard(user_color="#1a3a2a", assistant_color="#3a1a2a"))

config = ChatAppConfig(
    app_title="Context-Aware Assistant",
    initial_messages_factory=lambda: [
        Message(role="Assistant", content="Hello! I remember our conversation history.")
    ],
)

app = ChatApp(
    session_manager=custom_session_manager,
    chat_service=chat_service,
    renderer=renderer,
    config=config,
)

print("✅ Created fully customized DI chat app")

✅ Created fully customized DI chat app


## Testing Individual Components

In [10]:
# Test session management
manager = InMemorySessionManager()
session = manager.create_session("test-123")
session.add_message("User", "Hello")
print(f"Session messages: {session.get_message_dicts()}")

Session messages: [{'role': 'User', 'content': 'Hello', 'id': 'aa7212ac-7c89-4b42-9668-e462b6d73e89'}]


In [11]:
# Test chat service
from pylogue.service import ChatService

service = ChatService(responder=example_responder)
response = await service.process_message("Test message")
print(f"Service response: {response}")

Service response: Echo: Test message


In [12]:
# Test renderer with messages
from fasthtml.jupyter import render_ft

render_ft()

messages = [
    Message(role="User", content="Hello!"),
    Message(role="Assistant", content="Hi there! How can I help?"),
    Message(role="Assistant", content="", pending=True),
]

renderer = ChatRenderer()
renderer.render_messages(messages)

<div>
  <div id="chat-cards" class="chat-cards" style="display: flex; flex-direction: column; gap: 10px;">
    <div style="background: #272727; padding: 10px; font-size: 1.5em; width: 60%; align-self: center; text-align: right; border-radius: 1em; padding: 1.25em">
<span style="font-weight: bold; font-size: 1.1em; display: block; margin-bottom: 8px;"><u>🗣️ User: </u></span>      <div class="marked" style="white-space: pre-wrap;">Hello!</div>
    </div>
    <div style="background: #3B3B3B; padding: 10px; font-size: 1.5em; width: 60%; align-self: center; text-align: left; border-radius: 1em; padding: 1.25em">
<span style="font-weight: bold; font-size: 1.1em; display: block; margin-bottom: 8px;"><u>🕵️‍♂️ Assistant: </u></span>      <div class="marked" style="white-space: pre-wrap;">Hi there! How can I help?</div>
    </div>
    <div style="background: #3B3B3B; padding: 10px; font-size: 1.5em; width: 60%; align-self: center; text-align: left; border-radius: 1em; padding: 1.25em">
🕵️‍♂️ Assistant: <span class="spinner"></span>    </div>
  </div>
<script>if (window.htmx) htmx.process(document.body)</script></div>


## Running the App

To run the chat app in production:

```python
# Create your app
app = create_default_chat_app(responder=your_responder)

# Run it
app.run(host="0.0.0.0", port=5001)
```

Or get the FastHTML app for advanced usage:

```python
fasthtml_app = app.get_app()
# Now you can add custom routes, middleware, etc.
```